In [1]:
import pandas as pd
import numpy as np

In [2]:
filename = '../../data/processed/na-data-int.csv'
df = pd.read_csv(filename)

In [3]:
df.head()

,year,zipcode,EQI_zip,SFR_zip,RECPI_zip,EQI_MSA,SFR_MSA,RECPI_MSA,EQI_state,SFR_state,RECPI_state
0,1988,1001,0.000815,48.0,0.039108,0.001021,1235.0,1.260888,0.001476,17558.0,25.921940
1,1989,1001,0.001116,44.0,0.049100,0.001168,1049.0,1.225384,0.001751,15343.0,26.866861
2,1990,1001,0.001629,45.0,0.073317,0.001243,841.0,1.045161,0.001857,13556.0,25.172453
3,1991,1001,0.000826,27.0,0.022298,0.001375,714.0,0.981724,0.001823,12798.0,23.330479
4,1992,1001,0.002216,22.0,0.048744,0.001549,760.0,1.176877,0.002111,13289.0,28.052156


In [4]:
def pivot_df(df):
    five_years = [[2016,2011],[2011,2006],[2006,2001],[2001,1996],[1996,1991]]
    combined = pd.DataFrame()
    for y in five_years:
        print(y[0], y[1])
        mask = (df['year'] > y[1]) & (df['year'] <= y[0])
        subset = df.loc[mask]
        df['year_label'] = pd.cut(subset['year'], 5, labels=['year_1', 'year_2', 'year_3', 'year_4', 'year_5'])
        subset = df.loc[mask]
        melted = subset.melt(id_vars=['zipcode', 'year_label'])
        pivoted = pd.pivot_table(melted, values='value', index=['zipcode'], columns=['year_label', 'variable'])
        pivoted.columns = ['_'.join(col).strip() for col in pivoted.columns.values]
        pivoted = pivoted.drop(['year_2_year','year_3_year','year_4_year','year_5_year'], axis=1)
        pivoted.rename(columns={'year_1_year':'start_year'}, inplace=True)
        pivoted = pivoted.reset_index()
        pivoted['prediction_start_year'] = pivoted['start_year'] - 5
        pivoted['avg_five_years'] = pivoted.filter(regex='EQI_zip$', axis=1).mean(axis=1)
        combined = combined.append(pivoted)
    return combined


In [5]:
pivoted = pivot_df(df)

2016 2011
2011 2006
2006 2001
2001 1996
1996 1991


In [6]:
pivoted.head()

,zipcode,year_1_EQI_MSA,year_1_EQI_state,year_1_EQI_zip,year_1_RECPI_MSA,year_1_RECPI_state,year_1_RECPI_zip,year_1_SFR_MSA,year_1_SFR_state,year_1_SFR_zip,...,year_5_EQI_state,year_5_EQI_zip,year_5_RECPI_MSA,year_5_RECPI_state,year_5_RECPI_zip,year_5_SFR_MSA,year_5_SFR_state,year_5_SFR_zip,prediction_start_year,avg_five_years
0,1001,0.000743,0.001822,0.000546,1.184285,48.527607,0.030041,1594.0,26629.0,55.0,...,0.001729,0.000447,1.632499,61.478992,0.029936,1852.0,35562.0,67.0,2007.0,0.000898
1,1002,0.000743,0.001822,0.000974,1.184285,48.527607,0.064305,1594.0,26629.0,66.0,...,0.001729,0.005833,1.632499,61.478992,0.431667,1852.0,35562.0,74.0,2007.0,0.002022
2,1005,0.000192,0.001822,0.005553,0.041344,48.527607,0.033315,215.0,26629.0,6.0,...,0.001729,0.000605,0.059796,61.478992,0.004236,291.0,35562.0,7.0,2007.0,0.001500
3,1007,0.000743,0.001822,0.000366,1.184285,48.527607,0.011725,1594.0,26629.0,32.0,...,0.001729,0.000394,1.632499,61.478992,0.014573,1852.0,35562.0,37.0,2007.0,0.000412
4,1008,0.000743,0.001822,0.000630,1.184285,48.527607,0.001261,1594.0,26629.0,2.0,...,0.001729,0.000417,1.632499,61.478992,0.001252,1852.0,35562.0,3.0,2007.0,0.000386


In [7]:
pivoted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85170 entries, 0 to 17033
Data columns (total 49 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   zipcode                85170 non-null  int64  
 1   year_1_EQI_MSA         85170 non-null  float64
 2   year_1_EQI_state       85170 non-null  float64
 3   year_1_EQI_zip         85170 non-null  float64
 4   year_1_RECPI_MSA       85170 non-null  float64
 5   year_1_RECPI_state     85170 non-null  float64
 6   year_1_RECPI_zip       85170 non-null  float64
 7   year_1_SFR_MSA         85170 non-null  float64
 8   year_1_SFR_state       85170 non-null  float64
 9   year_1_SFR_zip         85170 non-null  float64
 10  start_year             85170 non-null  float64
 11  year_2_EQI_MSA         85170 non-null  float64
 12  year_2_EQI_state       85170 non-null  float64
 13  year_2_EQI_zip         85170 non-null  float64
 14  year_2_RECPI_MSA       85170 non-null  float64
 15  ye

In [8]:
y = pivoted[['zipcode','prediction_start_year','avg_five_years']]
y = y.rename(columns={'prediction_start_year':'start_year', 'avg_five_years':'avg_eqi_year_5-10'})
y.head()

,zipcode,start_year,avg_eqi_year_5-10
0,1001,2007.0,0.000898
1,1002,2007.0,0.002022
2,1005,2007.0,0.001500
3,1007,2007.0,0.000412
4,1008,2007.0,0.000386


In [9]:
X = pivoted.drop(['prediction_start_year','avg_five_years'], axis=1)
X.head()

,zipcode,year_1_EQI_MSA,year_1_EQI_state,year_1_EQI_zip,year_1_RECPI_MSA,year_1_RECPI_state,year_1_RECPI_zip,year_1_SFR_MSA,year_1_SFR_state,year_1_SFR_zip,...,year_4_SFR_zip,year_5_EQI_MSA,year_5_EQI_state,year_5_EQI_zip,year_5_RECPI_MSA,year_5_RECPI_state,year_5_RECPI_zip,year_5_SFR_MSA,year_5_SFR_state,year_5_SFR_zip
0,1001,0.000743,0.001822,0.000546,1.184285,48.527607,0.030041,1594.0,26629.0,55.0,...,70.0,0.000881,0.001729,0.000447,1.632499,61.478992,0.029936,1852.0,35562.0,67.0
1,1002,0.000743,0.001822,0.000974,1.184285,48.527607,0.064305,1594.0,26629.0,66.0,...,72.0,0.000881,0.001729,0.005833,1.632499,61.478992,0.431667,1852.0,35562.0,74.0
2,1005,0.000192,0.001822,0.005553,0.041344,48.527607,0.033315,215.0,26629.0,6.0,...,7.0,0.000205,0.001729,0.000605,0.059796,61.478992,0.004236,291.0,35562.0,7.0
3,1007,0.000743,0.001822,0.000366,1.184285,48.527607,0.011725,1594.0,26629.0,32.0,...,36.0,0.000881,0.001729,0.000394,1.632499,61.478992,0.014573,1852.0,35562.0,37.0
4,1008,0.000743,0.001822,0.000630,1.184285,48.527607,0.001261,1594.0,26629.0,2.0,...,3.0,0.000881,0.001729,0.000417,1.632499,61.478992,0.001252,1852.0,35562.0,3.0


In [10]:
data = pd.merge(left=y, right=X, on=['zipcode','start_year'])
data['start_year'] = data['start_year'].astype('int64')
data.head()

,zipcode,start_year,avg_eqi_year_5-10,year_1_EQI_MSA,year_1_EQI_state,year_1_EQI_zip,year_1_RECPI_MSA,year_1_RECPI_state,year_1_RECPI_zip,year_1_SFR_MSA,...,year_4_SFR_zip,year_5_EQI_MSA,year_5_EQI_state,year_5_EQI_zip,year_5_RECPI_MSA,year_5_RECPI_state,year_5_RECPI_zip,year_5_SFR_MSA,year_5_SFR_state,year_5_SFR_zip
0,1001,2007,0.000898,0.000702,0.001817,0.000611,1.081845,45.171143,0.036063,1541.0,...,55.0,0.000808,0.001884,0.000480,1.144370,45.149307,0.024020,1417.0,23961.0,50.0
1,1002,2007,0.002022,0.000702,0.001817,0.000895,1.081845,45.171143,0.068894,1541.0,...,59.0,0.000808,0.001884,0.001899,1.144370,45.149307,0.121545,1417.0,23961.0,64.0
2,1005,2007,0.001500,0.000176,0.001817,0.000654,0.060114,45.171143,0.006539,341.0,...,13.0,0.001136,0.001884,0.000531,0.235161,45.149307,0.004775,207.0,23961.0,9.0
3,1007,2007,0.000412,0.000702,0.001817,0.000507,1.081845,45.171143,0.021308,1541.0,...,21.0,0.000808,0.001884,0.000625,1.144370,45.149307,0.016257,1417.0,23961.0,26.0
4,1008,2007,0.000386,0.000702,0.001817,0.000280,1.081845,45.171143,0.001678,1541.0,...,3.0,0.000808,0.001884,0.000444,1.144370,45.149307,0.000444,1417.0,23961.0,1.0


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68136 entries, 0 to 68135
Data columns (total 48 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   zipcode             68136 non-null  int64  
 1   start_year          68136 non-null  int64  
 2   avg_eqi_year_5-10   68136 non-null  float64
 3   year_1_EQI_MSA      68136 non-null  float64
 4   year_1_EQI_state    68136 non-null  float64
 5   year_1_EQI_zip      68136 non-null  float64
 6   year_1_RECPI_MSA    68136 non-null  float64
 7   year_1_RECPI_state  68136 non-null  float64
 8   year_1_RECPI_zip    68136 non-null  float64
 9   year_1_SFR_MSA      68136 non-null  float64
 10  year_1_SFR_state    68136 non-null  float64
 11  year_1_SFR_zip      68136 non-null  float64
 12  year_2_EQI_MSA      68136 non-null  float64
 13  year_2_EQI_state    68136 non-null  float64
 14  year_2_EQI_zip      68136 non-null  float64
 15  year_2_RECPI_MSA    68136 non-null  float64
 16  year

In [12]:
data = data.sort_values(['zipcode','start_year']).reset_index(drop=True)

In [13]:
data.head()

,zipcode,start_year,avg_eqi_year_5-10,year_1_EQI_MSA,year_1_EQI_state,year_1_EQI_zip,year_1_RECPI_MSA,year_1_RECPI_state,year_1_RECPI_zip,year_1_SFR_MSA,...,year_4_SFR_zip,year_5_EQI_MSA,year_5_EQI_state,year_5_EQI_zip,year_5_RECPI_MSA,year_5_RECPI_state,year_5_RECPI_zip,year_5_SFR_MSA,year_5_SFR_state,year_5_SFR_zip
0,1001,1992,0.001287,0.001549,0.002111,0.002216,1.176877,28.052156,0.048744,760.0,...,23.0,0.001361,0.002326,0.001971,1.203223,38.430626,0.057166,884.0,16520.0,29.0
1,1001,1997,0.001853,0.001233,0.002386,0.000840,1.112023,41.174500,0.035283,902.0,...,39.0,0.001299,0.003029,0.001835,1.177120,55.327602,0.053203,906.0,18265.0,29.0
2,1001,2002,0.000602,0.001494,0.002450,0.001504,1.746593,50.713380,0.054131,1169.0,...,43.0,0.000917,0.001913,0.003813,1.454710,47.060886,0.183047,1586.0,24597.0,48.0
3,1001,2007,0.000898,0.000702,0.001817,0.000611,1.081845,45.171143,0.036063,1541.0,...,55.0,0.000808,0.001884,0.000480,1.144370,45.149307,0.024020,1417.0,23961.0,50.0
4,1002,1992,0.002931,0.001549,0.002111,0.000959,1.176877,28.052156,0.020148,760.0,...,26.0,0.001361,0.002326,0.001732,1.203223,38.430626,0.072728,884.0,16520.0,42.0


In [14]:
path = '../../data/processed/reshaped.csv'
data.to_csv(path, index=False)